In [2]:
import numpy
import os, ctypes
from scipy import integrate, LowLevelCallable
%run "integration_utils.py"

In [4]:
y = numpy.matrix('1 1; 1 -1; -1 1; -1 -1', dtype=float)
q = numpy.array([0.5, 0.2, 0.2, 0.1])

lib = ctypes.CDLL(os.path.abspath('std_gaussian.so'))
lib.f.restype = ctypes.c_double
lib.f.argtypes = (ctypes.c_int, ctypes.POINTER(ctypes.c_double))
p1 = LowLevelCallable(lib.f)
lib.g.restype = ctypes.c_double
lib.g.argtypes = (ctypes.c_int, ctypes.POINTER(ctypes.c_double), ctypes.c_void_p)
p3 = []
for j in range(len(y)):
    user_data = ctypes.cast(ctypes.pointer((ctypes.c_double*2)(y[j,0], y[j,1])), ctypes.c_void_p)
    p3.append(LowLevelCallable(lib.g, user_data))

In [5]:
def g_w_one_source(v, q, y, pdf, normpdf):
    n = v.size
    w = numpy.zeros(n)
    g = v.dot(q)
    for j in range(n):
        interval = gen_intervals(j, v, y)
        integral1 = integrate_over_intervals(interval, pdf)
        integral2 = integrate_over_intervals(interval, normpdf[j])
        w[j] = q[j] - integral1
        g += integral2 - v[j] * integral1
    return g, w

In [6]:
def subgradient_1(q, y, max_step, pdf, normpdf):
    v = numpy.zeros(len(y))
    v_best = v
    g_best = - numpy.inf
    for step in range(1, max_step):
        g, w = g_w_one_source(v, q, y, pdf, normpdf)
        if g > g_best:
            v_best = v
            g_best = g
        norm = numpy.linalg.norm(w)
        if norm < 1e-8:
            break
        alpha = 1 / step
        v += alpha * w / norm
        print(step, g_best, norm)
    return g_best, v_best

In [7]:
%prun subgradient_1(q, y, 20, p1, p3)

1 0.8084617567885385 0.30000000000000004
2 1.0506394672302237 0.18554490033301
3 1.1294821099256276 0.1306433520855199


KeyboardInterrupt: 

In [9]:
y = numpy.matrix('1 0; 0 1; -1 0', dtype=float)

In [10]:
g_w_one_source(numpy.array([0, 0, 0]), 
               numpy.array([0.33333337, 0.66666663, 0]),
               y,
               p1,
               p3)

(3.7663050227841532, array([-0.04166663,  0.41666663, -0.375     ]))